In [4]:
from IPython.display import SVG, display
import chess.svg 
import cairosvg
import os
import subprocess




def depict_position(g, at_ply=10):
    board = chess.Board()
    nm = 0
    for move in g.mainline_moves():
        nm = nm + 1
        if (move is not None):
            board.push(move)
        if (nm == at_ply):
            return board
    return board    

def save_positions_as_png(g, dir_name):
    
    if not os.path.exists(dir_name):
        os.makedirs(dir_name)
    else:
        for the_file in os.listdir(dir_name):
            file_path = os.path.join(dir_name, the_file)
            try:
                if os.path.isfile(file_path):
                    os.unlink(file_path)
            except Exception as e:
                print(e)

    board = chess.Board()
    nm = 0
    for move in g.mainline_moves():
        nm = nm + 1
        if (move is not None):
            board.push(move)
            svg_pos = chess.svg.board(board=board, size=400, coordinates=False)
            cairosvg.svg2png(bytestring=svg_pos,write_to=dir_name + 'output' + str(nm) + '.png')
            
# delay: centh of second, so 100=1 sec
#### produce a GIF of a game
def mkGIF(game, dir_final_result, ng, delay=50):
    gif_filename = "../" + dir_final_result + "game" + str(ng) + ".gif"
    dir_name = "game/" # TODO: temporary directory for hosting images per position
    save_positions_as_png(game, dir_name)
    os.chdir(dir_name)
    # create gif and move it to dir_final_result
    # two versions: one with ffmeg/convert; other with convert
    # cmd='cat `ls *.png | sort -V` | ffmpeg -i - outputf.gif; convert -delay ' + str(delay) + ' outputf.gif output.gif' + '; mv output.gif ' + gif_filename
    cmd='convert $(for a in `ls *.png | sort -V`; do printf -- "-delay ' + str(delay) +' %s " $a; done; ) output.gif' + '; mv output.gif ' + gif_filename 
    print(cmd)
    try: 
        p = subprocess.Popen(['/bin/bash', '-c', cmd])  
        (output, err) = p.communicate()  
        #This makes the wait possible
        p_status = p.wait()
        print("ouput:", output, "error:", err, "status:", p_status)
    except Exception as e:
        print(e)     
    os.chdir('..')
    
def analyze_game_best_score_evolution(g, move_time=100):
    score_evolution = pd.DataFrame(columns=['ply', 'score'])
    board = chess.Board()
    nm = 0
    white_turn = True
    engine = mkChessUCIEngine()
    engine.setoption({'MultiPV': 2})          
    for move in g.mainline_moves():        
        if (move is not None):
            board.push(move)
            # analyze move
            info_handler = chess.uci.InfoHandler()   
            engine.info_handlers.append(info_handler)
            # set position
            engine.position(board)
            bm = engine.go(movetime=move_time)   
            pvs = info_handler.info['pv']
            scores = info_handler.info['score']
            best_score = scores[1].cp
            if white_turn:
                best_score = best_score * -1
            score_evolution.loc[nm] = (nm, best_score)
            white_turn = not white_turn
            nm = nm + 1
        else:
            return score_evolution
        
    return score_evolution

def process_score_evo(score_evo, game):
    # difference's score at each move
    sdiff = [0] # no difference for first move    
    for i in range(1, len(score_evo) - 1):
        sdiff.append(abs(score_evo.loc[i]['score'] - score_evo.loc[i + 1]['score']))
    sdiff.append(0) # no difference for last move 
    
   
    score_diff = score_evo.copy()
    score_diff['diff'] = sdiff       
    max_score_pos = score_diff.loc[score_diff['diff'].idxmax()]
    
    plt.figure()
    score_diff.plot(x="ply", y="score", title="game" + str(ng), style="-", color="blue")
 
    
    
    score_diff_highest_ply = max_score_pos['ply']
    score_diff_highest = max_score_pos['diff']
    print("Highest score diff", score_diff_highest, "at ply", score_diff_highest_ply)
    b = depict_position(g=game, at_ply=score_diff_highest_ply)   
    display(SVG(chess.svg.board(board=b, size=300)))
    
    # score differential of AZ 
    whiteplayer = game.headers.get("White") == 'AlphaZero' # whether AZ has white piece
    azdiff = []
    for iply in range(0, len(score_diff)):
        # white turn and AZ is white, so it's AZ turn
        # or black turn and AZ is black, so it's AZ turn
        if (iply % 2 == 0 and whiteplayer) or (iply % 2 == 1 and not whiteplayer): 
            azdiff.append((score_diff.loc[iply]['diff'], score_diff.loc[iply]['score'], score_diff.loc[iply]['ply']))
    score_diff_AZ = pd.DataFrame(azdiff, columns=["diff", "score", "ply"])
    max_score_pos_AZ = score_diff_AZ.loc[score_diff_AZ['diff'].idxmax()]
    if (max_score_pos['ply'] == max_score_pos_AZ['ply']):
        print("AZ changes SF mind!")
        plt.scatter(max_score_pos_AZ['ply'], max_score_pos_AZ['score'], color="red", marker="X") 
    else:
        plt.scatter(max_score_pos['ply'], max_score_pos['score'], color="red")     
    
    
    plt.show()   
            
        
########## plot score evolution with Stockfish
### we mark with a red cross the most significant "change" 
### (basically to identify when Stockfish changes its mind... or when AlphaZero does make a difference)

### warning: can take a while, since we analyze all games with Stockfish
pgn = open(pgnfile)
dir_final_result = "gifs/"
ng = 1
while True:
    game = chess.pgn.read_game(pgn)
    if game is None:
        break
    # you can create a GIF of a game
    # mkGIF(game, dir_final_result, ng)    
    score_evo = analyze_game_best_score_evolution(game, move_time=2000)  ### you can modify the amount of time given to Stockfish
    process_score_evo(score_evo, game)
    ng = ng + 1
    if (ng == 20):
        break
 # GIF of the first game
ng = 1
pgn = open("longest.pgn")
game = chess.pgn.read_game(pgn)
mkGIF(game, dir_final_result, ng)       

ModuleNotFoundError: No module named 'cairosvg'